In [ ]:
import requests
import zipfile
import pandas as pd

def access_static_gtfs(url: str) -> bytes:
    response = requests.get(url)
    response.raise_for_status()
    return response.content

def extract_static_gtfs(zip: bytes, table: str = "stops.txt") -> pd.DataFrame:
    with zipfile.ZipFile(io.BytesIO(zip)) as zf:
        table_data = zf.read(table)  # Returns bytes

    table_df = pd.read_csv(io.BytesIO(table_data))
    return table_df

url = "https://rrgtfsfeeds.s3.amazonaws.com/gtfs_subway.zip"
zip_bytes = access_static_gtfs(url)



In [110]:
import io

with zipfile.ZipFile(io.BytesIO(zip_bytes)) as zf:
    print(zf.namelist())

stop_times = extract_static_gtfs(zip_bytes, table="stop_times.txt")
shapes = extract_static_gtfs(zip_bytes, table="shapes.txt")
stops = extract_static_gtfs(zip_bytes, table="stops.txt")
transfers = extract_static_gtfs(zip_bytes, table="transfers.txt")

['agency.txt', 'calendar_dates.txt', 'calendar.txt', 'routes.txt', 'shapes.txt', 'stop_times.txt', 'stops.txt', 'transfers.txt', 'trips.txt']


In [124]:
(
    stop_times
    .assign(
        id=lambda df: df["trip_id"].str.extract(r'_(.*?)$')
    )
)

,trip_id,stop_id,arrival_time,departure_time,stop_sequence,id
0,AFA25GEN-1038-Sunday-00_000600_1..S03R,101S,00:06:00,00:06:00,1,000600_1..S03R
1,AFA25GEN-1038-Sunday-00_000600_1..S03R,103S,00:07:30,00:07:30,2,000600_1..S03R
2,AFA25GEN-1038-Sunday-00_000600_1..S03R,104S,00:09:00,00:09:00,3,000600_1..S03R
3,AFA25GEN-1038-Sunday-00_000600_1..S03R,106S,00:10:30,00:10:30,4,000600_1..S03R
4,AFA25GEN-1038-Sunday-00_000600_1..S03R,107S,00:12:00,00:12:00,5,000600_1..S03R
...,...,...,...,...,...,...
562592,SIR-FA2017-SI017-Weekday-08_147100_SI..N03R,S27N,25:03:00,25:03:00,17,147100_SI..N03R
562593,SIR-FA2017-SI017-Weekday-08_147100_SI..N03R,S28N,25:06:00,25:06:00,18,147100_SI..N03R
562594,SIR-FA2017-SI017-Weekday-08_147100_SI..N03R,S29N,25:08:00,25:08:00,19,147100_SI..N03R
562595,SIR-FA2017-SI017-Weekday-08_147100_SI..N03R,S30N,25:10:00,25:10:00,20,147100_SI..N03R


In [100]:
(shapes)

,shape_id,shape_pt_sequence,shape_pt_lat,shape_pt_lon
0,1..N03R,0,40.702068,-74.013664
1,1..N03R,1,40.703199,-74.014792
2,1..N03R,2,40.703226,-74.014820
3,1..N03R,3,40.703253,-74.014846
4,1..N03R,4,40.703280,-74.014870
...,...,...,...,...
149829,SI..S07R,685,40.513696,-74.250493
149830,SI..S07R,686,40.513579,-74.250706
149831,SI..S07R,687,40.513458,-74.250917
149832,SI..S07R,688,40.513334,-74.251124


In [102]:
(
    shapes
    .merge(
        stops.loc[lambda df: df["location_type"].eq(1)], left_on=["shape_pt_lat", "shape_pt_lon"], right_on=["stop_lat", "stop_lon"], how="left"
    )
    .loc[lambda df: df["stop_id"].notnull(), :]
)

,shape_id,shape_pt_sequence,shape_pt_lat,shape_pt_lon,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
0,1..N03R,0,40.702068,-74.013664,142,South Ferry,40.702068,-74.013664,1.0,NaN
34,1..N03R,34,40.707513,-74.013783,139,Rector St,40.707513,-74.013783,1.0,NaN
40,1..N03R,40,40.711835,-74.012188,138,WTC Cortlandt,40.711835,-74.012188,1.0,NaN
77,1..N03R,77,40.715478,-74.009266,137,Chambers St,40.715478,-74.009266,1.0,NaN
94,1..N03R,94,40.719318,-74.006886,136,Franklin St,40.719318,-74.006886,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
149797,SI..S07R,526,40.533674,-74.191794,S16,Huguenot,40.533674,-74.191794,1.0,NaN
149813,SI..S07R,542,40.525507,-74.200064,S15,Prince's Bay,40.525507,-74.200064,1.0,NaN
149860,SI..S07R,589,40.522410,-74.217847,S14,Pleasant Plains,40.522410,-74.217847,1.0,NaN
149892,SI..S07R,621,40.519631,-74.229141,S13,Richmond Valley,40.519631,-74.229141,1.0,NaN


In [69]:
(
    stop_times
    .loc[lambda df: df["trip_id"].str.contains("_6")]
)

,trip_id,stop_id,arrival_time,departure_time,stop_sequence
133396,AFA25GEN-6031-Sunday-00_000700_6..S01R,601S,00:07:00,00:07:00,1
133397,AFA25GEN-6031-Sunday-00_000700_6..S01R,602S,00:09:30,00:09:30,2
133398,AFA25GEN-6031-Sunday-00_000700_6..S01R,603S,00:11:00,00:11:00,3
133399,AFA25GEN-6031-Sunday-00_000700_6..S01R,604S,00:12:30,00:12:30,4
133400,AFA25GEN-6031-Sunday-00_000700_6..S01R,606S,00:13:30,00:13:30,5
...,...,...,...,...,...
177042,AFA25GEN-6091-Weekday-00_150300_6..N01R,606N,25:59:30,25:59:30,34
177043,AFA25GEN-6091-Weekday-00_150300_6..N01R,604N,26:00:30,26:00:30,35
177044,AFA25GEN-6091-Weekday-00_150300_6..N01R,603N,26:02:00,26:02:00,36
177045,AFA25GEN-6091-Weekday-00_150300_6..N01R,602N,26:03:30,26:03:30,37


In [ ]:
with duckdb.connect("mta.duckdb") as conn:
    df = conn.execute(
        """
    
        """
    ).fetchdf()

df.drop_duplicates("route_id")

,updated_at,trip_id,start_date,route_id,direction,location,location_status,headsign_text,departure_time,underway,train_assigned,last_position_update,current_stop_sequence_index,num_stops_left,has_delay_alert
0,2026-01-29 00:09:21,000650_1..S03R,2026-01-29,1,S,104S,INCOMING_AT,South Ferry,2026-01-29 00:06:30,True,True,2026-01-29 00:09:12,3.0,36,False
14,2026-01-29 00:09:21,144100_2..N08R,2026-01-29,2,N,241N,STOPPED_AT,Wakefield-241 St,2026-01-30 00:01:00,True,True,2026-01-29 00:08:39,7.0,55,False
32,2026-01-29 00:09:21,144300_3..N90R,2026-01-29,3,N,123N,STOPPED_AT,Harlem-148 St,2026-01-30 00:03:00,True,True,2026-01-29 00:09:03,3.0,8,False
38,2026-01-29 00:09:21,000100_4..S01X052,2026-01-29,4,S,409S,IN_TRANSIT_TO,149 St-Grand Concourse,2026-01-29 00:01:00,True,True,2026-01-29 00:09:20,7.0,7,False
59,2026-01-29 00:09:21,000200_5..S32R,2026-01-29,5,S,505S,STOPPED_AT,E 180 St,2026-01-29 00:02:00,True,True,2026-01-29 00:09:03,5.0,2,False
65,2026-01-29 00:09:21,144100_6..N01R,2026-01-29,6,N,634N,IN_TRANSIT_TO,Pelham Bay Park,2026-01-30 00:01:00,True,True,2026-01-29 00:08:41,7.0,32,False
84,2026-01-29 00:09:21,000350_7..N,2026-01-29,7,N,721N,IN_TRANSIT_TO,Flushing-Main St,2026-01-29 00:03:30,True,True,2026-01-29 00:09:16,5.0,18,True
110,2026-01-29 00:09:21,009750_7..S,2026-01-28,7X,S,720S,STOPPED_AT,Times Sq-42 St,2026-01-28 01:37:30,True,True,2026-01-28 01:57:30,5.0,5,False
113,2026-01-29 00:09:21,000000_GS.N01R,2026-01-29,GS,N,NaN,NaN,Times Sq-42 St,2026-01-29 00:00:00,False,False,NaT,NaN,2,False
123,2026-01-29 00:09:21,133600_A..S74R,2026-01-28,A,S,H10S,STOPPED_AT,Far Rockaway-Mott Av,2026-01-28 22:16:00,True,True,2026-01-29 00:04:39,53.0,1,False


In [ ]:
duckdb.sql(
    """
    SELECT
        trip_id,
        REGEXP_EXTRACT(trip_id, '_([0-9A-Z]+)\.+', 1) AS route
    FROM stop_times
    where
        route is null
    """
)

┌─────────┬─────────┐
│ trip_id │  route  │
│ varchar │ varchar │
├─────────┴─────────┤
│      0 rows       │
└───────────────────┘

In [63]:
duckdb.sql(
    """
    SELECT distinct
        REGEXP_EXTRACT(trip_id, '_([0-9A-Z]+)\.+', 1) AS route
    FROM stop_times
    """
).fetchdf().sort_values(by = "route")

,route
8,1
11,2
21,3
23,4
16,5
17,6
14,7
7,A
15,B
4,C


In [ ]:
(
    stop_times
    .assign(
        route=lambda df: df['trip_id'].str.extract(r'_([A-Z]+)_')[0]
    )
)

,trip_id,stop_id,arrival_time,departure_time,stop_sequence
0,AFA25GEN-1038-Sunday-00_000600_1..S03R,101S,00:06:00,00:06:00,1
1,AFA25GEN-1038-Sunday-00_000600_1..S03R,103S,00:07:30,00:07:30,2
2,AFA25GEN-1038-Sunday-00_000600_1..S03R,104S,00:09:00,00:09:00,3
3,AFA25GEN-1038-Sunday-00_000600_1..S03R,106S,00:10:30,00:10:30,4
4,AFA25GEN-1038-Sunday-00_000600_1..S03R,107S,00:12:00,00:12:00,5
...,...,...,...,...,...
562592,SIR-FA2017-SI017-Weekday-08_147100_SI..N03R,S27N,25:03:00,25:03:00,17
562593,SIR-FA2017-SI017-Weekday-08_147100_SI..N03R,S28N,25:06:00,25:06:00,18
562594,SIR-FA2017-SI017-Weekday-08_147100_SI..N03R,S29N,25:08:00,25:08:00,19
562595,SIR-FA2017-SI017-Weekday-08_147100_SI..N03R,S30N,25:10:00,25:10:00,20


In [ ]:
(
    stop_times
    .assign(
        route=
    )
)



['AFA25GEN-1038-Sunday-00_000600_1..S03R',
 'AFA25GEN-1038-Sunday-00_002600_1..S03R',
 'AFA25GEN-1038-Sunday-00_004600_1..S03R',
 'AFA25GEN-1038-Sunday-00_006600_1..S03R',
 'AFA25GEN-1038-Sunday-00_007200_1..N03R',
 'AFA25GEN-1038-Sunday-00_008600_1..S03R',
 'AFA25GEN-1038-Sunday-00_009200_1..N03R',
 'AFA25GEN-1038-Sunday-00_010600_1..S03R',
 'AFA25GEN-1038-Sunday-00_011200_1..N03R',
 'AFA25GEN-1038-Sunday-00_012600_1..S03R',
 'AFA25GEN-1038-Sunday-00_013200_1..N03R',
 'AFA25GEN-1038-Sunday-00_014600_1..S03R',
 'AFA25GEN-1038-Sunday-00_015200_1..N03R',
 'AFA25GEN-1038-Sunday-00_016600_1..S03R',
 'AFA25GEN-1038-Sunday-00_017200_1..N03R',
 'AFA25GEN-1038-Sunday-00_018600_1..S03R',
 'AFA25GEN-1038-Sunday-00_019200_1..N03R',
 'AFA25GEN-1038-Sunday-00_020600_1..S03R',
 'AFA25GEN-1038-Sunday-00_021200_1..N03R',
 'AFA25GEN-1038-Sunday-00_022600_1..S03R',
 'AFA25GEN-1038-Sunday-00_023200_1..N03R',
 'AFA25GEN-1038-Sunday-00_024600_1..S03R',
 'AFA25GEN-1038-Sunday-00_025200_1..N03R',
 'AFA25GEN-

In [31]:
routes_df

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,route_sort_order
0,A,MTA NYCT,A,8 Avenue Express,"Trains operate between Inwood-207 St, Manhatta...",1,https://www.mta.info/schedules/subway/a-train,0062CF,FFFFFF,1
1,C,MTA NYCT,C,8 Avenue Local,"Trains operate between 168 St, Manhattan, and ...",1,https://www.mta.info/schedules/subway/c-train,0062CF,FFFFFF,2
2,E,MTA NYCT,E,8 Avenue Local,Trains operate between Jamaica Center-Parsons/...,1,https://www.mta.info/schedules/subway/e-train,0062CF,FFFFFF,3
3,B,MTA NYCT,B,6 Avenue Express,"Trains operate, weekdays only, between 145 St,...",1,https://www.mta.info/schedules/subway/b-train,EB6800,FFFFFF,4
4,D,MTA NYCT,D,6 Avenue Express,"Trains operate, at all times, from Norwood-205...",1,https://www.mta.info/schedules/subway/d-train,EB6800,FFFFFF,5
5,F,MTA NYCT,F,Queens Blvd Express/6 Av Local,Trains operate at all times between Jamaica-17...,1,https://www.mta.info/schedules/subway/f-train,EB6800,FFFFFF,6
6,FX,MTA NYCT,FX,Brooklyn F Express,Trains operate rush hour only between Jamaica-...,1,https://www.mta.info/schedules/subway/f-train,EB6800,FFFFFF,7
7,M,MTA NYCT,M,Queens Blvd Local/6 Av Local,Trains operate weekdays between Middle Village...,1,https://www.mta.info/schedules/subway/m-train,EB6800,FFFFFF,8
8,G,MTA NYCT,G,Brooklyn-Queens Crosstown,"Trains operate between Court Sq, Queens and Ch...",1,https://www.mta.info/schedules/subway/g-train,799534,FFFFFF,9
9,J,MTA NYCT,J,Nassau St Local,Trains operate weekdays between Jamaica Center...,1,https://www.mta.info/schedules/subway/j-train,8E5C33,FFFFFF,10


In [1]:
import pandas as pd
import duckdb
import os

os.chdir("..")

In [15]:
with duckdb.connect("mta.duckdb") as conn:
    query = """
    from public.raw_stop_time_updates
    """
    stops = """
    from public.raw_stops
    """
    result_df = conn.execute(query).fetchdf()
    stops = conn.execute(stops).fetchdf()

In [11]:
(
    result_df
    .assign(parent=lambda df: df["stop_id"].str.slice(0, 3))
    .loc[lambda df: df["location_type"].isna() & df["parent_station"].ne(df["parent"]), :]
)

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station,parent


In [17]:
(
    result_df
    .merge(stops, how="left", on="stop_id")
)

,updated_at,trip_id,stop_id,arrival,departure,stop_name,stop_lat,stop_lon,location_type,parent_station
0,2026-01-29 00:09:21,000650_1..S03R,104S,2026-01-29 00:10:20,2026-01-29 00:10:20,231 St,40.878856,-73.904834,NaN,104
1,2026-01-29 00:09:21,000650_1..S03R,142S,2026-01-29 01:04:20,NaT,South Ferry,40.702068,-74.013664,NaN,142
2,2026-01-29 00:09:21,144900_1..N03R,139N,2026-01-29 00:10:36,2026-01-29 00:10:36,Rector St,40.707513,-74.013783,NaN,139
3,2026-01-29 00:09:21,144900_1..N03R,101N,2026-01-29 01:07:06,NaT,Van Cortlandt Park-242 St,40.889248,-73.898583,NaN,101
4,2026-01-29 00:09:21,145900_1..N03R,142N,NaT,2026-01-29 00:19:00,South Ferry,40.702068,-74.013664,NaN,142
...,...,...,...,...,...,...,...,...,...,...
635,2026-01-29 00:09:23,003600_SI.S03R,S09S,2026-01-29 01:18:00,2026-01-29 01:18:00,Tottenville,40.512764,-74.251961,NaN,S09
636,2026-01-29 00:09:23,006100_SI.N03R,S09N,2026-01-29 01:01:00,2026-01-29 01:01:00,Tottenville,40.512764,-74.251961,NaN,S09
637,2026-01-29 00:09:23,006100_SI.N03R,S31N,2026-01-29 01:43:00,2026-01-29 01:43:00,St George,40.643748,-74.073643,NaN,S31
638,2026-01-29 00:09:23,006600_SI.S03R,S31S,2026-01-29 01:06:00,2026-01-29 01:06:00,St George,40.643748,-74.073643,NaN,S31


In [25]:
(
    result_df
)

,updated_at,trip_id,stop_id,arrival,departure
0,2026-01-29 00:09:21,000650_1..S03R,104S,2026-01-29 00:10:20,2026-01-29 00:10:20
1,2026-01-29 00:09:21,000650_1..S03R,142S,2026-01-29 01:04:20,NaT
2,2026-01-29 00:09:21,144900_1..N03R,139N,2026-01-29 00:10:36,2026-01-29 00:10:36
3,2026-01-29 00:09:21,144900_1..N03R,101N,2026-01-29 01:07:06,NaT
4,2026-01-29 00:09:21,145900_1..N03R,142N,NaT,2026-01-29 00:19:00
...,...,...,...,...,...
635,2026-01-29 00:09:23,003600_SI.S03R,S09S,2026-01-29 01:18:00,2026-01-29 01:18:00
636,2026-01-29 00:09:23,006100_SI.N03R,S09N,2026-01-29 01:01:00,2026-01-29 01:01:00
637,2026-01-29 00:09:23,006100_SI.N03R,S31N,2026-01-29 01:43:00,2026-01-29 01:43:00
638,2026-01-29 00:09:23,006600_SI.S03R,S31S,2026-01-29 01:06:00,2026-01-29 01:06:00


In [ ]:
with duckdb.connect("mta.duckdb") as conn:
    query = """
    from public.raw_stop_time_updates
    """
    stops = """
    from public.raw_stops
    """
    result_df = conn.execute(query).fetchdf()
    stops = conn.execute(stops).fetchdf()